
### **Otimização Linear: Algoritmo Branch and Bound aplicado a problemas de alocação de Equipes de Enfermagem**
Projeto de TCC Referente ao curso de Engenharia Biomédica (ICT - UNIFESP)

Autor: Magno Gonçalves Fonseca
contato: mgfonseca1999@gmail.com

Este notebook demonstra a modelagem e solução de um problema de alocação de equipes de enfermagem usando otimização linear.
O objetivo é minimizar os custos operacionais ao mesmo tempo que respeitamos restrições como carga horária e cobertura mínima de profissionais.

Para saber se o funcionário $ e $ estará escalado no turno $ t $, definimos variáveis binárias que representem esse estado, onde:

$$
x_{e,t} = 
\begin{cases} 
1, & \text{se o enfermeiro } e \text{ está no turno } t, \\ 
0, & \text{caso contrário}.
\end{cases}
\quad \forall e, t
$$

Para cada enfermeiro $ e $ e conjunto de turnos simultâneos $ T_s $, impôs-se a seguinte restrição:

$$ \sum_{t \in T_s} x_{e,t} \leq 1. $$

Cada enfermeiro tem uma carga horária máxima semanal de 40 horas, definida como:

$$ \sum_{t} x_{e,t} \cdot D_t \leq 40, \quad \forall e $$

onde $ D_t $ representa a duração do turno $ t $ . Cada turno deve respeitar a quantidade mínima ($ P_{\text{min}} $) e máxima ($ P_{\text{max}} $) de enfermeiros alocados:

$$ P_{\text{min},t} \leq \sum_{e} x_{e,t} \leq P_{\text{max},t}, \quad \forall t. $$

Trabalhar em dois turnos consecutivos é permitido, desde que a soma da duração dos turnos não ultrapasse 10 horas:

$$ x_{e, t_x} + x_{e, t_y} \leq 1, \quad \text{se }(D_{t_x} + D_{t_y}) > 10. $$

$$ \forall \text{ par } (t_x, t_y) \text{ consecutivos}. $$

**Descanso diário:** deve-se garantir que entre turnos não consecutivos (separados por longos intervalos), exista um intervalo mínimo de descanso diário, como 12 horas:

$$ x_{e, t_a} + x_{e, t_b} \leq 1, \quad \text{se } \text{descanso}(t_a, t_b) < 12. $$

O objetivo do modelo foi minimizar o custo total da operação hospitalar. O custo para cada enfermeiro é calculado como o produto da duração dos turnos pelos seus salários por hora:

$$ \text{Custo}_e = \sum_{t} x_{e,t} \cdot D_t \cdot S_e $$

onde $ S_e $ é o salário por hora do enfermeiro $ e $ . O custo total foi, então:

$$ \text{Custo Total} = \sum_{e} \text{Custo}_e. $$


##### Passo 1 - Importar depedências

In [1]:
import pip
REQUIRED_MINIMUM_PANDAS_VERSION = '0.17.1'
try:
    import pandas as pd
    assert pd.__version__ >= REQUIRED_MINIMUM_PANDAS_VERSION
except:
    raise Exception("Version %s or above of Pandas is required to run this notebook" % REQUIRED_MINIMUM_PANDAS_VERSION)

In [2]:
CSS = """
body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}
    table.dataframe thead th:first-child {
        display: none;
    }
    table.dataframe tbody th {
        display: none;
    }
"""

In [3]:
from IPython.core.display import HTML
HTML('<style>{}</style>'.format(CSS))

from IPython.display import display

In [4]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [5]:
# This notebook requires pandas to work
import pandas as pd
from pandas import DataFrame

# Make sure that xlrd package, which is a pandas optional dependency, is installed
# This package is required for Excel I/O
try:
    import xlrd
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install xlrd 
    else:
        !pip install --user xlrd      

##### Passo 2 - Carregar os dados do problema para o DataFrame

In [6]:
# Dados do exemplo original
# data_url = "https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/nurses_data.xls?raw=true"


# Dados locais, com personalizações de volumes
file_path = "nurses_data.xls"

# Ler o arquivo Excel local
nurse_xls_file = pd.ExcelFile(file_path)

df_skills = nurse_xls_file.parse('Skills')
df_depts = nurse_xls_file.parse('Departments')
#df_shifts = nurse_xls_file.parse('Shifts_larger')
df_shifts = nurse_xls_file.parse('Shifts')
# Rename df_shifts index
df_shifts.index.name = 'shiftId'

# Index is column 0: name
#df_nurses = nurse_xls_file.parse('Nurses_larger', header=0, index_col=0)
df_nurses = nurse_xls_file.parse('Nurses', header=0, index_col=0)
df_nurse_skills = nurse_xls_file.parse('NurseSkills')
df_vacations = nurse_xls_file.parse('NurseVacations')
df_associations = nurse_xls_file.parse('NurseAssociations')
df_incompatibilities = nurse_xls_file.parse('NurseIncompatibilities')

# Display the nurses dataframe
print("#nurses = {}".format(len(df_nurses)))
print("#shifts = {}".format(len(df_shifts)))
print("#vacations = {}".format(len(df_vacations)))

#nurses = 65
#shifts = 84
#vacations = 59


In [7]:
# maximum work time (in hours)
max_work_time = 40

### Passo 3: Preparar os dados

Precisamos pré-computar dados adicionais para os turnos.  
Para cada turno, precisamos do horário de início e de término expressos em horas, contando a partir do início da semana: segunda-feira às 8h é convertido para 8, terça-feira às 8h é convertido para 24 + 8 = 32, e assim por diante.

####        Sub-passo 1
Começamos adicionando uma coluna extra chamada `dow` (dia da semana), que converte a string "dia" em um inteiro de 0 a 6 (segunda-feira é 0, domingo é 6).

In [8]:
days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
day_of_weeks = dict(zip(days, range(7)))

# utility to convert a day string e.g. "Monday" to an integer in 0..6
def day_to_day_of_week(day):
    return day_of_weeks[day.strip().lower()]

# for each day name, we normalize it by stripping whitespace and converting it to lowercase
# " Monday" -> "monday"
df_shifts["dow"] = df_shifts.day.apply(day_to_day_of_week)
df_shifts

,department,day,start_time,end_time,min_req,max_req,dow
shiftId,,,,,,,
0,Emergency,Monday,2,8,3,5,0
1,Emergency,Monday,8,12,4,7,0
2,Emergency,Monday,12,18,2,5,0
3,Emergency,Monday,18,2,3,7,0
4,Orthopedics,Monday,2,8,3,5,0
...,...,...,...,...,...,...,...
79,Orthopedics,Sunday,2,12,5,7,6
80,Orthopedics,Sunday,12,20,7,9,6
81,Orthopedics,Sunday,20,2,8,12,6


#### Sub-passo #2: Calcular o horário de início absoluto de cada turno.

Calcular o horário de início na semana é simples: basta adicionar `24 * dow` à coluna `start_time`. O resultado é armazenado em uma nova coluna chamada `wstart`.

In [9]:
df_shifts["wstart"] = df_shifts.start_time + 24 * df_shifts.dow

#### Sub-passo #3: Calcular o horário de término absoluto de cada turno.

Calcular o horário de término absoluto é um pouco mais complicado, pois alguns turnos se estendem até a meia-noite. Por exemplo, o Turno #3 começa na segunda-feira às 18:00 e termina na terça-feira às 2:00. O horário de término absoluto do Turno #3 é 26, não 2.  
A regra geral para calcular o horário de término absoluto é:

`abs_end_time = end_time + 24 * dow + (start_time >= end_time ? 24 : 0)`

Novamente, usamos *pandas* para adicionar uma nova coluna calculada chamada `wend`. Isso é feito usando o método `apply` do *pandas* com uma função anônima `lambda` aplicada às linhas. O parâmetro `raw=True` impede a criação de uma *Series* do *pandas* para cada linha, o que melhora significativamente o desempenho em conjuntos de dados grandes.

In [10]:
# an auxiliary function to calculate absolute end time of a shift
def calculate_absolute_endtime(start, end, dow):
    return 24*dow + end + (24 if start>=end else 0)

# store the results in a new column
df_shifts["wend"] = df_shifts.apply(lambda row: calculate_absolute_endtime(
        row.start_time, row.end_time, row.dow), axis=1)

#### Sub-passo #4: Calcular a duração de cada turno.

Calcular a duração de cada turno agora é uma diferença direta entre colunas. O resultado é armazenado na coluna `duration`.

In [11]:
df_shifts["duration"] = df_shifts.wend - df_shifts.wstart

#### Sub-passo #5: Calcular a demanda mínima para cada turno.

A demanda mínima é o produto da duração (em horas) pelo número mínimo de enfermeiros requeridos. Assim, em termos de horas de enfermeiro, essa demanda é armazenada em outra nova coluna chamada `min_demand`.

Por fim, exibimos o *DataFrame* de turnos atualizado com todas as colunas calculadas.

In [12]:
# also compute minimum demand in nurse-hours
df_shifts["min_demand"] = df_shifts.min_req * df_shifts.duration
# finally check the modified shifts dataframe
df_shifts

,department,day,start_time,end_time,min_req,max_req,dow,wstart,wend,duration,min_demand
shiftId,,,,,,,,,,,
0,Emergency,Monday,2,8,3,5,0,2,8,6,18
1,Emergency,Monday,8,12,4,7,0,8,12,4,16
2,Emergency,Monday,12,18,2,5,0,12,18,6,12
3,Emergency,Monday,18,2,3,7,0,18,26,8,24
4,Orthopedics,Monday,2,8,3,5,0,2,8,6,18
...,...,...,...,...,...,...,...,...,...,...,...
79,Orthopedics,Sunday,2,12,5,7,6,146,156,10,50
80,Orthopedics,Sunday,12,20,7,9,6,156,164,8,56
81,Orthopedics,Sunday,20,2,8,12,6,164,170,6,48


### Passo 4: Configurar o modelo

Declarar as variáveis e decisão binarias associadas a alocação dos turnos:
$$
x_{e,t} = 
\begin{cases} 
1, & \text{se o enfermeiro } e \text{ está no turno } t, \\ 
0, & \text{caso contrário}.
\end{cases}
\quad \forall e, t
$$

In [13]:
from pulp import LpProblem, LpVariable, lpSum, LpStatus, LpStatusOptimal, value, PULP_CBC_CMD, GLPK, CPLEX
import pulp

# Criando o problema de otimização
prob = pulp.LpProblem("nurses", pulp.LpMinimize)

# Definindo as coleções de enfermeiros e turnos
all_nurses = df_nurses.index.values  # Lista de enfermeiros
all_shifts = df_shifts.index.values  # Lista de turnos

# Definindo as variáveis binárias de atribuição
assigned = pulp.LpVariable.dicts(
    "assign", 
    [(nurse, shift) for nurse in all_nurses for shift in all_shifts], 
    cat='Binary'
)

Encontrar o conjunto de turnos simultâneos $ T_s $, para atribuir a primeira restrição:
$$ \sum_{t \in T_s} x_{e,t} \leq 1. $$

In [14]:
import pandasql as psql
import sqlite3

df = pd.DataFrame(df_shifts)

# Conectar ao banco de dados SQLite na memória
conn = sqlite3.connect(':memory:')

# Carregar o DataFrame em uma tabela SQL temporária
df.to_sql('shifts', conn, index=False, if_exists='replace')

# Consulta SQL para encontrar turnos que ocorrem no mesmo horário
query = """
SELECT group_concat(shiftId) as turnos_simultaneos
FROM df
GROUP BY wstart 
HAVING COUNT(*) > 1
"""

# Executar a consulta no DataFrame
turnos_simultaneos = psql.sqldf(query, locals())

# Exibir o resultado
print(turnos_simultaneos)

   turnos_simultaneos
0                 0,4
1      1,5,8,10,12,14
2      2,6,9,11,13,15
3                 3,7
4               16,20
5   17,21,24,26,28,31
6   18,22,25,27,29,32
7         19,23,30,33
8               34,38
9         35,39,42,44
10        36,40,43,45
11              37,41
12              46,50
13        47,51,54,56
14        48,52,55,57
15              49,53
16              58,62
17        59,63,66,68
18        60,64,67,69
19              61,65
20              70,73
21              71,74
22              72,75
23              76,79
24           77,80,83
25              78,81


In [15]:
for nurse in all_nurses:
    #Adicionar a restrição para cada enfermeiro e grupo de turnos simultâneos
    for _, row in turnos_simultaneos.iterrows():
        # Pegar os turnos simultâneos e transformá-los em uma lista de inteiros
        simultaneous_shifts = [int(shift_id) for shift_id in row['turnos_simultaneos'].split(',')]
        # Criar a restrição: a soma das variáveis de decisão desses turnos deve ser <= 1 para cada enfermeiro
        prob += lpSum([assigned[(nurse, shift)] for shift in simultaneous_shifts]) <= 1, f"RestrSimult_{nurse}_{simultaneous_shifts}"

Adicionar a retrição de quantidade maxima trabalhada na semana:
$$ \sum_{t} x_{e,t} \cdot D_t \leq 40, \quad \forall e $$

In [16]:
for nurse in all_nurses:
    #Adicionar a restrição de 40 horas trabalhadas na semana
    prob += lpSum([assigned[(nurse, shift)]*df_shifts['duration'][shift] for shift in all_shifts]) <= 40, f"Worktime_{nurse}"

Adicionar a restrição responsável por demonstrar a quantidade mínima e maxima de funcionários para cada turno:
$$ P_{\text{min},t} \leq \sum_{e} x_{e,t} \leq P_{\text{max},t}, \quad \forall t. $$

In [17]:
for shift in all_shifts:
    #Adicionar a restrição de quantidade mínima e maxima
    prob += lpSum([assigned[(nurse, shift)] for nurse in all_nurses]) >= df_shifts['min_req'][shift], f"Min_Req_{shift}"
    prob += lpSum([assigned[(nurse, shift)] for nurse in all_nurses]) <= df_shifts['max_req'][shift], f"Max_Req_{shift}"

Adicionar as retrições de turnos consecutivos e descanso minimo

$$ x_{e, t_x} + x_{e, t_y} \leq 1, \quad \text{se }(D_{t_x} + D_{t_y}) > 10. $$
$$ \forall \text{ par } (t_x, t_y) \text{ consecutivos}. $$
$$ x_{e, t_a} + x_{e, t_b} \leq 1, \quad \text{se } \text{descanso}(t_a, t_b) < 12. $$


In [18]:
from itertools import combinations
# Lista de pares de turnos no mesmo dia para restrição
max_consecutive_pairs = []
min_rest_pairs = []

for shift1, shift2 in combinations(all_shifts, 2):
    day1 = df_shifts.loc[shift1, 'day']
    day2 = df_shifts.loc[shift2, 'day']
    start_time1 = df_shifts.loc[shift1, 'start_time']
    end_time1 = df_shifts.loc[shift1, 'end_time']
    start_time2 = df_shifts.loc[shift2, 'start_time']
    end_time2 = df_shifts.loc[shift2, 'end_time']

    # Verificar se os turnos são no mesmo dia
    if day1 == day2:
        # Turnos consecutivos (shift2 começa logo após shift1 terminar)
        if start_time2 >= end_time1:
            work_hours = end_time1 - start_time1 + (end_time2 - start_time2)
            if work_hours > 10:
                max_consecutive_pairs.append((shift1, shift2))
        # Turnos não consecutivos (verificar descanso mínimo diário)
        rest_time = start_time2 - end_time1
        if rest_time > 0 and rest_time < 12:  # Menos de 12h de descanso
            min_rest_pairs.append((shift1, shift2))


In [19]:
# Adicionar restrição para trabalho consecutivo máximo de 10h
for nurse in all_nurses:
    for shift1, shift2 in max_consecutive_pairs:
        prob += (
            assigned[(nurse, shift1)] + assigned[(nurse, shift2)]
        ) <= 1, f"MaxConsecutive_{nurse}_{shift1}_{shift2}"

In [20]:
# Adicionar restrição para descanso mínimo diário de 12h
for nurse in all_nurses:
    for shift1, shift2 in min_rest_pairs:
        prob += (
            assigned[(nurse, shift1)] + assigned[(nurse, shift2)]
        ) <= 1, f"MinRest_{nurse}_{shift1}_{shift2}"

Adicionando função objetivo relacionada ao custo da operação:
$$ \text{Custo}_e = \sum_{t} x_{e,t} \cdot D_t \cdot S_e $$

onde $ S_e $ é o salário por hora do enfermeiro $ e $ . portanto, a função objetivo é demonstrada como:

$$ \text{Minimize   } \sum_{e} \text{Custo}_e. $$

In [21]:
CustoTotal = 0
for nurse in all_nurses:
    CustoEnfermeira = 0
    for shift in all_shifts:
        CustoEnfermeira += assigned[(nurse, shift)] * df_shifts['duration'][shift] * df_nurses.loc[nurse, 'pay_rate']
    CustoTotal += CustoEnfermeira

prob += CustoTotal, f"CustoTotal"

Mostrar o problema completo

In [22]:
print(prob)

nurses:
MINIMIZE
138*assign_('Alice',_np.int64(0)) + 92*assign_('Alice',_np.int64(1)) + 92*assign_('Alice',_np.int64(10)) + 138*assign_('Alice',_np.int64(11)) + 92*assign_('Alice',_np.int64(12)) + 138*assign_('Alice',_np.int64(13)) + 92*assign_('Alice',_np.int64(14)) + 138*assign_('Alice',_np.int64(15)) + 138*assign_('Alice',_np.int64(16)) + 92*assign_('Alice',_np.int64(17)) + 138*assign_('Alice',_np.int64(18)) + 184*assign_('Alice',_np.int64(19)) + 138*assign_('Alice',_np.int64(2)) + 138*assign_('Alice',_np.int64(20)) + 92*assign_('Alice',_np.int64(21)) + 138*assign_('Alice',_np.int64(22)) + 184*assign_('Alice',_np.int64(23)) + 92*assign_('Alice',_np.int64(24)) + 138*assign_('Alice',_np.int64(25)) + 92*assign_('Alice',_np.int64(26)) + 138*assign_('Alice',_np.int64(27)) + 92*assign_('Alice',_np.int64(28)) + 138*assign_('Alice',_np.int64(29)) + 184*assign_('Alice',_np.int64(3)) + 184*assign_('Alice',_np.int64(30)) + 92*assign_('Alice',_np.int64(31)) + 138*assign_('Alice',_np.int64(32)) 

Enviar modelagem ao solucionador:

In [23]:
import time;

resultado = []

inicio = time.time()
prob.solve()
fim = time.time()
tempo_execucao = fim - inicio
resultado_objetivo = prob.objective.value()

resultado.append({
"Solver": "PULP_CBC_CMD",
"Tempo de Execução (s)": tempo_execucao,
"Valor da Função Objetivo": resultado_objetivo,
"Status": prob.status
})


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/magnofonseca/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/083db551f0e0482fa7b2c6de603f6880-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/083db551f0e0482fa7b2c6de603f6880-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16228 COLUMNS
At line 82984 RHS
At line 99208 BOUNDS
At line 104669 ENDATA
Problem MODEL has 16223 rows, 5460 columns and 50375 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 61536 - 0.35 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 11085 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10712 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8884 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6739 strengthened rows, 0 substitutions
Cgl0003I 0 fi

In [24]:
prob.objective.value()

np.float64(61536.0)

In [25]:
# Preparação dos resultados para visualização
resultados = []
for nurse in all_nurses:
    for shift in all_shifts:
        if value(assigned[(nurse, shift)]) == 1:  # Se a enfermeira foi alocada ao turno
            custo_turno = df_shifts['duration'][shift] * df_nurses.loc[nurse, 'pay_rate']
            resultados.append([nurse, shift, df_shifts['duration'][shift], df_nurses.loc[nurse, 'pay_rate'], custo_turno])

# Cria DataFrame para exibir os resultados
df_resultados = pd.DataFrame(resultados, columns=['Enfermeira', 'Turno', 'Duração (h)', 'Taxa por Hora', 'Custo Turno'])
df_resultados.loc['Total'] = ['', '', '', '', value(prob.objective)]


Manter o resultado em um SqLite, afim de realizar consultas e filtros para a análise dos resultados

In [26]:
# import sqlite3
# 
# # Cria uma conexão com o banco de dados em memória
# conn = sqlite3.connect(":memory:")
# 
# # Carrega o DataFrame como uma tabela SQL
# df_resultados.to_sql("resultados", conn, index=False, if_exists="replace")
# 
# # 1. Carga horária máxima semanal de 40 horas
# query_carga_horaria = """
# SELECT Enfermeira, SUM([Duração (h)]) AS total_horas
# FROM resultados
# GROUP BY Enfermeira
# HAVING total_horas > 40
# """
# resultado_carga_horaria = pd.read_sql_query(query_carga_horaria, conn)
# 
# # 2. Trabalho consecutivo máximo de 10 horas
# query_consecutivo = """
# SELECT a.Enfermeira, a.Turno as Turno1, a.[Hora Início], a.[Hora Fim],
#        b.Turno as Turno2, b.[Hora Início], b.[Hora Fim],
#        (a.[Duração (h)] + b.[Duração (h)]) as Total_Horas_Trabalho
# FROM resultados a
# JOIN resultados b 
#   ON a.Enfermeira = b.Enfermeira 
#  AND a.[Dia da Semana] = b.[Dia da Semana]
#  AND a.[Hora Fim] <= b.[Hora Início]
# WHERE Total_Horas_Trabalho > 10
# """
# resultado_consecutivo = pd.read_sql_query(query_consecutivo, conn)
# 
# # 3. Descanso mínimo diário de 12 horas
# query_descanso_minimo = """
# SELECT a.Enfermeira, a.Turno as Turno1, a.[Hora Fim], 
#        b.Turno as Turno2, b.[Hora Início], 
#        (b.[Hora Início] - a.[Hora Fim]) as Descanso_Horas
# FROM resultados a
# JOIN resultados b
#   ON a.Enfermeira = b.Enfermeira
#  AND a.[Dia da Semana] = b.[Dia da Semana]
#  AND a.[Hora Fim] <= b.[Hora Início]
# WHERE Descanso_Horas < 12
# """
# resultado_descanso = pd.read_sql_query(query_descanso_minimo, conn)
# 
# # 4. Mínimo e máximo de enfermeiros por turno
# query_min_max_enfermeiros = """
# SELECT Turno, COUNT(Enfermeira) as Total_Enfermeiras,
#        Min_Req, Max_Req
# FROM resultados
# GROUP BY Turno
# HAVING Total_Enfermeiras < Min_Req OR Total_Enfermeiras > Max_Req
# """
# resultado_min_max = pd.read_sql_query(query_min_max_enfermeiros, conn)
# 
# # Exibir resultados
# print("Violação de carga horária máxima semanal:")
# print(resultado_carga_horaria)
# 
# print("\nViolação de trabalho consecutivo máximo de 10 horas:")
# print(resultado_consecutivo)
# 
# print("\nViolação de descanso mínimo diário de 12 horas:")
# print(resultado_descanso)
# 
# print("\nViolação de mínimo e máximo de enfermeiros por turno:")
# print(resultado_min_max)
# # Fecha a conexão
# conn.close()

Exportar resultado para uma planilha

In [27]:
df_resultados.to_excel("resultado_32_42.xlsx", index=False)

print("Resultados exportados para 'resultado_alocacao_enfermeiras.xlsx'")

Resultados exportados para 'resultado_alocacao_enfermeiras.xlsx'
